In [61]:
import pickle
import joblib
import time
import os 

import pandas as pd
import numpy as np

from numpy import mean, std
from datetime import timedelta
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, KBinsDiscretizer, PolynomialFeatures

# HERE YOU HAVE TO CHOOSE BPI_Challenge_2012 or BPI_Challenge_2017
chosed_dataset = 'BPI_Challenge_2012'

# RFC for next event prediction

In [62]:
# extract the 1-3 lags
df = pd.read_csv(f'../data/{chosed_dataset}_naive.csv')

df_train = pd.read_csv(f"../data/train_{chosed_dataset}_naive.csv")
df_test = pd.read_csv(f"../data/test_{chosed_dataset}_naive.csv")

# for train data 
df_train['concept:name - lag_1'] = df_train.groupby('case:concept:name')['concept:name'].shift(1).fillna('absent')
df_train['concept:name - lag_2'] = df_train.groupby('case:concept:name')['concept:name'].shift(2).fillna('absent')
df_train['concept:name - lag_3'] = df_train.groupby('case:concept:name')['concept:name'].shift(3).fillna('absent')

# define target
df_train['next concept:name'] = df_train.groupby('case:concept:name')['concept:name'].shift(-1).fillna('absent')

# now for test data
df_test['concept:name - lag_1'] = df_test.groupby('case:concept:name')['concept:name'].shift(1).fillna('absent')
df_test['concept:name - lag_2'] = df_test.groupby('case:concept:name')['concept:name'].shift(2).fillna('absent')
df_test['concept:name - lag_3'] = df_test.groupby('case:concept:name')['concept:name'].shift(3).fillna('absent')

# define target
df_test['next concept:name'] = df_test.groupby('case:concept:name')['concept:name'].shift(-1).fillna('absent')


# extract for the whole dataframe to apply the model in future
# for train data 
df['concept:name - lag_1'] = df.groupby('case:concept:name')['concept:name'].shift(1).fillna('absent')
df['concept:name - lag_2'] = df.groupby('case:concept:name')['concept:name'].shift(2).fillna('absent')
df['concept:name - lag_3'] = df.groupby('case:concept:name')['concept:name'].shift(3).fillna('absent')

# define target
df['next concept:name'] = df.groupby('case:concept:name')['concept:name'].shift(-1).fillna('absent')


In [63]:
# only for testing 
# total_rows = df_2017.shape[0]

# # Calculate the index to divide the DataFrame 
# split_index = total_rows // 2

# # Split the DataFrame into two halves
# df_2017 = df_2017.iloc[:split_index, :]

df

,Unnamed: 0.1,Unnamed: 0,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ,start_timestamp,time:timestamp diff,next time:timestamp,next time:timestamp naive,next concept:name naive,concept:name - lag_1,concept:name - lag_2,concept:name - lag_3,next concept:name
0,0,0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546,01:10:2011 00:38:44.546000,173688,20000,2011-10-01 00:38:44.546000+00:00,NaN,2011-10-01 00:38:44.880,2011-10-01 00:38:44.816,A_PARTLYSUBMITTED,absent,absent,absent,A_PARTLYSUBMITTED
1,1,1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880,01:10:2011 00:38:44.546000,173688,20000,2011-10-01 00:38:44.880000+00:00,0 days 00:00:00.334000,2011-10-01 00:39:37.906,2011-10-01 00:39:21.880,A_PREACCEPTED,A_SUBMITTED,absent,absent,A_PREACCEPTED
2,2,2,112.0,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906,01:10:2011 00:38:44.546000,173688,20000,2011-10-01 00:39:37.906000+00:00,0 days 00:00:53.026000,2011-10-01 00:39:38.875,2011-10-01 00:39:38.286,W_Completeren aanvraag,A_PARTLYSUBMITTED,A_SUBMITTED,absent,W_Completeren aanvraag
3,3,3,112.0,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875,01:10:2011 00:38:44.546000,173688,20000,2011-10-01 00:39:38.875000+00:00,0 days 00:00:00.969000,2011-10-01 11:36:46.437,2011-10-01 00:44:56.025,W_Completeren aanvraag,A_PREACCEPTED,A_PARTLYSUBMITTED,A_SUBMITTED,W_Completeren aanvraag
4,4,26,112.0,COMPLETE,A_SUBMITTED,2011-10-01 08:08:58.256,01:10:2011 08:08:58.256000,173691,5000,2011-10-01 08:08:58.256000+00:00,NaN,2011-10-01 08:09:02.195,2011-10-01 08:08:58.526,A_PARTLYSUBMITTED,absent,absent,absent,A_PARTLYSUBMITTED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261441,261441,256158,10863.0,START,W_Nabellen incomplete dossiers,2012-03-14 15:59:28.309,27:02:2012 14:12:41.868000,213276,15000,2012-03-14 15:59:28.309000+00:00,0 days 00:22:26.421000,2012-03-14 16:00:09.680,2012-03-14 16:04:52.709,W_Nabellen offertes,W_Nabellen incomplete dossiers,W_Nabellen incomplete dossiers,W_Nabellen incomplete dossiers,W_Nabellen incomplete dossiers
261442,261442,256159,10863.0,COMPLETE,W_Nabellen incomplete dossiers,2012-03-14 16:00:09.680,27:02:2012 14:12:41.868000,213276,15000,2012-03-14 16:00:09.680000+00:00,0 days 00:00:41.371000,NaN,2012-03-14 16:05:34.080,NaN,W_Nabellen incomplete dossiers,W_Nabellen incomplete dossiers,W_Nabellen incomplete dossiers,absent
261443,261443,235011,11169.0,START,W_Nabellen offertes,2012-03-14 16:02:03.883,15:02:2012 10:10:36.503000,209595,13000,2012-03-14 16:02:03.883000+00:00,5 days 19:12:30.382000,NaN,2012-03-14 16:04:41.903,NaN,W_Nabellen offertes,W_Nabellen offertes,W_Nabellen offertes,absent
261444,261444,246922,11203.0,START,W_Nabellen incomplete dossiers,2012-03-14 16:04:46.192,21:02:2012 23:38:40.044000,211624,35000,2012-03-14 16:04:46.192000+00:00,0 days 00:08:38.193000,2012-03-14 16:04:54.681,2012-03-14 16:10:10.592,W_Nabellen offertes,W_Nabellen incomplete dossiers,W_Nabellen incomplete dossiers,W_Valideren aanvraag,W_Nabellen incomplete dossiers


In [64]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'org:resource', 'lifecycle:transition',
       'concept:name', 'time:timestamp', 'case:REG_DATE', 'case:concept:name',
       'case:AMOUNT_REQ', 'start_timestamp', 'time:timestamp diff',
       'next time:timestamp', 'next time:timestamp naive',
       'next concept:name naive', 'concept:name - lag_1',
       'concept:name - lag_2', 'concept:name - lag_3', 'next concept:name'],
      dtype='object')

In [65]:
# Prepare data
X_train = df_train[['concept:name - lag_2', 'concept:name - lag_1', 'concept:name']]
X_test = df_test[['concept:name - lag_2', 'concept:name - lag_1', 'concept:name']]

y_train = df_train[['next concept:name']]
y_test = df_test[['next concept:name']]

# Use LabelEncoder to convert string labels to numerical values
label_encoder = LabelEncoder()
X_train['concept:name - lag_1'] = label_encoder.fit_transform(X_train['concept:name - lag_1'])
X_train['concept:name - lag_2'] = label_encoder.fit_transform(X_train['concept:name - lag_2'])
X_train['concept:name'] = label_encoder.fit_transform(X_train['concept:name'])
y_train = label_encoder.fit_transform(y_train)

X_test['concept:name - lag_1'] = label_encoder.fit_transform(X_test['concept:name - lag_1'])
X_test['concept:name - lag_2'] = label_encoder.fit_transform(X_test['concept:name - lag_2'])
X_test['concept:name'] = label_encoder.fit_transform(X_test['concept:name'])
y_test = label_encoder.fit_transform(y_test)

df['concept:name - lag_1'] = label_encoder.fit_transform(df['concept:name - lag_1'])
df['concept:name - lag_2'] = label_encoder.fit_transform(df['concept:name - lag_2'])
df['concept:name'] = label_encoder.fit_transform(df['concept:name'])

/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_77714/1536737238.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['concept:name - lag_1'] = label_encoder.fit_transform(X_train['concept:name - lag_1'])
/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_77714/1536737238.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['concept:name - lag_2'] = label_encoder.fit_transform(X_train['concept:name - lag_2'])
/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_77714

In [66]:

# Define the Random Forest Classifier
rf_clf = RandomForestClassifier(n_jobs=-1)

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [21, 22, 23],
    'max_depth': [18, 19, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform nested cross-validation
inner_cv = KFold(n_splits=5, shuffle=False, random_state=None)  # 5-fold inner cross-validation

grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, scoring='accuracy', cv=inner_cv)
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Print the results
print(f"Best params: {grid_search.best_params_}")
print(f'Nested Cross-Validation Accuracy: {grid_search.best_score_:.4f}')
print(best_model.score(X_test, y_test))


Best params: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 22}
Nested Cross-Validation Accuracy: 0.8262
0.6984118567224329


In [73]:
X = df[['concept:name - lag_2', 'concept:name - lag_1', 'concept:name' ]]

df['next concept:name rfc'] = label_encoder.inverse_transform(best_model.predict(X))

In [74]:
df = df.drop(['concept:name - lag_1', 'concept:name - lag_2', 'concept:name - lag_3'], axis=1, errors='ignore')

In [75]:
df

,Unnamed: 0.1,Unnamed: 0,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ,start_timestamp,time:timestamp diff,next time:timestamp,next time:timestamp naive,next concept:name naive,next concept:name,next concept:name rfc
0,0,0,112.0,COMPLETE,9,2011-10-01 00:38:44.546,01:10:2011 00:38:44.546000,173688,20000,2011-10-01 00:38:44.546000+00:00,NaN,2011-10-01 00:38:44.880,2011-10-01 00:38:44.816,A_PARTLYSUBMITTED,A_PARTLYSUBMITTED,A_PARTLYSUBMITTED
1,1,1,112.0,COMPLETE,6,2011-10-01 00:38:44.880,01:10:2011 00:38:44.546000,173688,20000,2011-10-01 00:38:44.880000+00:00,0 days 00:00:00.334000,2011-10-01 00:39:37.906,2011-10-01 00:39:21.880,A_PREACCEPTED,A_PREACCEPTED,O_SENT_BACK
2,2,2,112.0,COMPLETE,7,2011-10-01 00:39:37.906,01:10:2011 00:38:44.546000,173688,20000,2011-10-01 00:39:37.906000+00:00,0 days 00:00:53.026000,2011-10-01 00:39:38.875,2011-10-01 00:39:38.286,W_Completeren aanvraag,W_Completeren aanvraag,W_Beoordelen fraude
3,3,3,112.0,SCHEDULE,19,2011-10-01 00:39:38.875,01:10:2011 00:38:44.546000,173688,20000,2011-10-01 00:39:38.875000+00:00,0 days 00:00:00.969000,2011-10-01 11:36:46.437,2011-10-01 00:44:56.025,W_Completeren aanvraag,W_Completeren aanvraag,W_Beoordelen fraude
4,4,26,112.0,COMPLETE,9,2011-10-01 08:08:58.256,01:10:2011 08:08:58.256000,173691,5000,2011-10-01 08:08:58.256000+00:00,NaN,2011-10-01 08:09:02.195,2011-10-01 08:08:58.526,A_PARTLYSUBMITTED,A_PARTLYSUBMITTED,A_PARTLYSUBMITTED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261441,261441,256158,10863.0,START,20,2012-03-14 15:59:28.309,27:02:2012 14:12:41.868000,213276,15000,2012-03-14 15:59:28.309000+00:00,0 days 00:22:26.421000,2012-03-14 16:00:09.680,2012-03-14 16:04:52.709,W_Nabellen offertes,W_Nabellen incomplete dossiers,W_Completeren aanvraag
261442,261442,256159,10863.0,COMPLETE,20,2012-03-14 16:00:09.680,27:02:2012 14:12:41.868000,213276,15000,2012-03-14 16:00:09.680000+00:00,0 days 00:00:41.371000,NaN,2012-03-14 16:05:34.080,NaN,absent,W_Completeren aanvraag
261443,261443,235011,11169.0,START,21,2012-03-14 16:02:03.883,15:02:2012 10:10:36.503000,209595,13000,2012-03-14 16:02:03.883000+00:00,5 days 19:12:30.382000,NaN,2012-03-14 16:04:41.903,NaN,absent,W_Nabellen incomplete dossiers
261444,261444,246922,11203.0,START,20,2012-03-14 16:04:46.192,21:02:2012 23:38:40.044000,211624,35000,2012-03-14 16:04:46.192000+00:00,0 days 00:08:38.193000,2012-03-14 16:04:54.681,2012-03-14 16:10:10.592,W_Nabellen offertes,W_Nabellen incomplete dossiers,W_Completeren aanvraag


In [70]:
df.to_csv(f'../data/{chosed_dataset}_rfc_xgboost.csv')

In [71]:
# Save the best model to a file using pickle
# Create the folder for model weights
os.makedirs('../model_weights', exist_ok=True)
model_filename = '../model_weights/random_forest_model.pkl'

with open(model_filename, 'wb') as model_file:
    pickle.dump(best_model, model_file)

### XGBoost for time till next event prediction

In [77]:
df = pd.read_csv(f'../data/{chosed_dataset}_rfc_xgboost.csv')

# find out which columns we have to preprocess
num_columns = df.select_dtypes(include=[np.number]).columns
num_imputer = SimpleImputer(strategy='median')
df[num_columns] = num_imputer.fit_transform(df[num_columns])

# Impute missing values for categorical columns
cat_columns = df.select_dtypes(include=['object', 'category']).columns
cat_imputer = SimpleImputer(strategy='most_frequent', fill_value='missing')
df[cat_columns] = cat_imputer.fit_transform(df[cat_columns])

# Encoding categorical variables
label_encoders = {}
for column in cat_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Calculate correlation for numerical features
correlations = df[num_columns].corrwith(df['time:timestamp diff']).sort_values(ascending=False)
print("Correlations with the target:\n", correlations)

# UNCOMMENT BELLOW CODE  FOR MORE INFO ABOUT DATA

# # Prepare df_2017 for mutual information calculation
# # Drop the target variable and ensure no NaN values exist
# X = df_2017.drop(columns=['time_to_next_event_seconds'])
# y = df_2017['time_to_next_event_seconds']

# # Calculate mutual information
# mi_scores = mutual_info_regression(X, y, random_state=0)
# mi_scores = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)
# print("\nMutual Information Scores:\n", mi_scores)


Correlations with the target:
 org:resource         0.359202
concept:name         0.322481
Unnamed: 0.2         0.050627
Unnamed: 0.1         0.050627
case:AMOUNT_REQ      0.019240
Unnamed: 0          -0.006164
case:concept:name   -0.006184
dtype: float64


In [ ]:
X = df.drop(['case:concept:name', 'Unnamed: 0', 'EventOrigin',
       'EventID', 'case:LoanGoal', 'case:ApplicationType', 'case:concept:name', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'Selected', 'CreditScore', 'OfferedAmount', 'OfferID', 'predicted_time_to_next_event_seconds'], axis=1, errors='ignore')

y = df['time_to_next_event_seconds']  # Target for prediction

X_train = pd.read_csv(f'../data/train_{chosed_dataset}_rfc_xgboost.csv')
X_test = pd.read_csv(f'../data/test_{chosed_dataset}_rfc_xgboost.csv')

y_train = pd.read_csv(f'../data/train_{chosed_dataset}_rfc_xgboost.csv')
y_test = pd.read_csv(f'../data/test_{chosed_dataset}_rfc_xgboost.csv')

# Preprocessing for numerical and categorical columns
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ])

# Fit the preprocessor on the training data and transform both training and validation data
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Initialize the model
model = XGBRegressor(objective='reg:squarederror')

# Fit the model using the transformed training set and validate using the transformed validation set
model.fit(X_train_transformed, y_train)

# Transform the test set and predict
X_test_transformed = preprocessor.transform(X_test)
y_pred_test = model.predict(X_test_transformed)

# Save the best model to a file using pickle
model_filename = '../model_weights/xgboost_time.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(best_model, model_file)

# Evaluate the model on the test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f'Test RMSE: {rmse_test}')

r2_score_value = r2_score(y_test, y_pred_test)
print(f'R² score: {r2_score_value}')
# For predicting and adding new columns, you need to transform the whole dataset
X_transformed = preprocessor.transform(X)

# predict the outcomes for entire dataset
df['predicted_time_to_next_event_seconds_XG'] = np.abs(model.predict(X_transformed).round(1))

# decode labels
for column in label_encoders:
    print(column)
    le = label_encoders[column]
    print(le)
    df_2017[column] = le.inverse_transform(df_2017[column])

# drop unused columns
df_2017 = df_2017.drop(['elapsed_time_from_start', 'predicted_start_time', 'timestamp_difference', 'event_seq', 'Unnamed: 0.1', 'Unnamed: 0', 'time_to_next_event'], axis=1, errors='ignore')

# convert in proper timedeltas
df_2017['predicted_time_to_next_event_seconds_XG'] = df_2017['predicted_time_to_next_event_seconds_XG'].apply(lambda x: timedelta(seconds=x))
df_2017['time_to_next_event_seconds'] = df_2017['time_to_next_event_seconds'].apply(lambda x: timedelta(seconds=x))
df_2017['predicted_time_to_next_event_seconds'] = df_2017['predicted_time_to_next_event_seconds'].apply(lambda x: timedelta(seconds=x))

# return the naive model predictions on its place
df_2017['predicted_step'] = df_2017['predicted_step'].shift(-1)

# proper naming for columns
df_2017.rename(columns={'time_to_next_event_seconds': 'time_to_next_event'}, inplace=True, errors='ignore')
df_2017.rename(columns={'predicted_time_to_next_event_seconds': 'time_to_next_event_baseline'}, inplace=True,  errors='ignore')
df_2017.rename(columns={'predicted_time_to_next_event_seconds_XG': 'time_to_next_event_xgboost'}, inplace=True,  errors='ignore')
df_2017.rename(columns={'predicted_step': 'concept:name next naive'}, inplace=True,  errors='ignore')

/Users/alexraudvee/Desktop/TU_e/year_2/Q3Y2_Process_Mining/Process_Mining_1/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Running time for model: 1.7901618480682373 seconds
Test RMSE: 20920.835001461575
R² score: 0.9984493382777463
Action
LabelEncoder()
org:resource
LabelEncoder()
concept:name
LabelEncoder()
EventOrigin
LabelEncoder()
EventID
LabelEncoder()
lifecycle:transition
LabelEncoder()
time:timestamp
LabelEncoder()
case:LoanGoal
LabelEncoder()
case:ApplicationType
LabelEncoder()
case:concept:name
LabelEncoder()
Accepted
LabelEncoder()
Selected
LabelEncoder()
OfferID
LabelEncoder()
timestamp_difference
LabelEncoder()
time_to_next_event
LabelEncoder()
elapsed_time_from_start
LabelEncoder()
predicted_start_time
LabelEncoder()
predicted_step
LabelEncoder()
concept:name next
LabelEncoder()
concept:name next rfc
LabelEncoder()


In [ ]:
df_2017

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,...,Selected,CreditScore,OfferedAmount,OfferID,time_to_next_event,time_to_next_event_baseline,concept:name next naive,concept:name next,concept:name next rfc,time_to_next_event_xgboost
0,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392,Existing loan takeover,New credit,Application_652823628,...,True,0.0,15000.0,Offer_1000145087,0 days 00:00:00.010000,0 days 00:00:35.170000,W_Complete application,W_Complete application,W_Complete application,0 days 00:00:48.900002
1,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403,Existing loan takeover,New credit,Application_652823628,...,True,0.0,15000.0,Offer_1000145087,0 days 00:00:00.010000,0 days 00:00:00.010000,A_Concept,A_Concept,A_Concept,0 days 00:00:48.900002
2,statechange,User_1,A_Concept,Application,ApplState_642383566,complete,2016-01-01 09:52:36.413,Existing loan takeover,New credit,Application_652823628,...,True,0.0,15000.0,Offer_1000145087,4 days 23:34:08.080000,2 days 01:50:47.740000,A_Create Application,A_Create Application,A_Create Application,4 days 22:35:25
3,Created,User_1,A_Create Application,Application,Application_1691306052,complete,2016-01-01 10:16:11.500,Home improvement,New credit,Application_1691306052,...,True,0.0,15000.0,Offer_1000145087,0 days 00:00:00.050000,0 days 00:00:00.040000,A_Submitted,A_Submitted,A_Submitted,0 days 00:00:48.900002
4,statechange,User_1,A_Submitted,Application,ApplState_284636842,complete,2016-01-01 10:16:11.549,Home improvement,New credit,Application_1691306052,...,True,0.0,15000.0,Offer_1000145087,0 days 00:00:00.190000,0 days 00:00:00.300000,W_Handle leads,W_Handle leads,W_Handle leads,0 days 00:00:48.900002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201081,statechange,User_99,A_Pending,Application,ApplState_1437687321,complete,2017-02-01 09:58:43.970,Remaining debt home,New credit,Application_654932562,...,True,0.0,15000.0,Offer_1000145087,0 days 00:00:00,0 days 00:00:00,W_Call incomplete files,W_Validate application,W_Validate application,0 days 00:00:48.900002
1201082,Deleted,User_99,W_Validate application,Workflow,Workitem_1690791617,ate_abort,2017-02-01 09:58:43.972,Remaining debt home,New credit,Application_654932562,...,True,0.0,15000.0,Offer_1000145087,0 days 00:00:00.010000,0 days 00:00:00.020000,W_Call incomplete files,O_Cancelled,W_Validate application,0 days 00:00:48.900002
1201083,statechange,User_99,O_Cancelled,Offer,OfferState_2080870299,complete,2017-02-01 09:58:43.981,Remaining debt home,New credit,Application_654932562,...,True,0.0,15000.0,Offer_1237672415,0 days 00:00:00.010000,0 days 00:00:00.010000,W_Call incomplete files,O_Cancelled,W_Validate application,0 days 00:00:48.900002
1201084,statechange,User_99,O_Cancelled,Offer,OfferState_1140159487,complete,2017-02-01 09:58:43.988,Remaining debt home,New credit,Application_654932562,...,True,0.0,15000.0,Offer_1471775802,0 days 00:00:00.010000,0 days 00:00:00.010000,W_Call incomplete files,O_Cancelled,W_Validate application,0 days 00:00:48.900002


In [ ]:
df_2017.to_csv('BPI_2017_rfc_xgboost.csv', index=False)

## Code below is RFC train file

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Convert the importances into a DataFrame
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=True)

# Plotting feature importances
plt.figure(figsize=(12, 8))
barplot = sns.barplot(x='Importance', y='Feature', data=feature_importance_df)

plt.title('Feature Importance', fontsize=20)
plt.xlabel('Importance', fontsize=18)
plt.ylabel('Feature', fontsize=18)

# Increase tick label size
barplot.xaxis.set_tick_params(labelsize=16)
barplot.yaxis.set_tick_params(labelsize=16)

plt.show()
plt.savefig('feature.png')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Convert the importances into a DataFrame
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=True)

# Plotting feature importances
plt.figure(figsize=(12, 8))
barplot = sns.barplot(x='Importance', y='Feature', data=feature_importance_df)

plt.title('Feature Importance', fontsize=20)
plt.xlabel('Importance', fontsize=18)
plt.ylabel('Feature', fontsize=18)

# Increase tick label size
barplot.xaxis.set_tick_params(labelsize=16)
barplot.yaxis.set_tick_params(labelsize=16)

plt.show()
plt.savefig('feature.png')

In [ ]:
predicted_counts = df_2017['concept:name next rfc'].value_counts()

plt.figure(figsize=(10, 7))
sns.barplot(x=predicted_counts.index, y=predicted_counts.values)
plt.title('Predicted Next Step Distribution')
plt.xlabel('Next Steps')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

In [ ]:
class_names = label_encoder.inverse_transform(range(n_classes))

# Compute ROC curve and ROC area for each class
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plotting
plt.figure(figsize=(10, 7))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'ROC curve of {class_names[i]} (area = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve with True Class Names')
plt.legend(loc="lower right", prop={'size': 6})  # Adjust size as needed
plt.show()


In [ ]:
# Compute precision-recall curve and average precision for each class
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_test_binarized[:, i],
                                                        y_pred_proba[:, i])
    average_precision[i] = average_precision_score(y_test_binarized[:, i], y_pred_proba[:, i])

# Plotting
plt.figure(figsize=(10, 7))
for i in range(n_classes):
    plt.plot(recall[i], precision[i], label=f'{class_names[i]} (area = {average_precision[i]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve with True Class Names')
plt.legend(loc="best", prop={'size': 6})  # Adjust size as needed
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assume 'average_precision' is a dictionary containing the AP scores for each class
# where keys are numerical class indices and values are AP scores.

# Get class names from the label encoder
class_names = label_encoder.inverse_transform(sorted(average_precision.keys()))

# Get AP scores in the order of class names
ap_scores = [average_precision[i] for i in sorted(average_precision.keys())]

# Create a bar plot
plt.figure(figsize=(12, 8))
bars = plt.bar(range(len(class_names)), ap_scores, color='skyblue')

# Add the AP scores above the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval, round(yval, 2), va='bottom')  # va: vertical alignment

plt.xlabel('Class Names')
plt.ylabel('Average Precision Score')
plt.title('Average Precision Scores by Class')
plt.xticks(range(len(class_names)), class_names, rotation=90)  # Rotate class names for better visibility
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Assuming 'y_test' are your true labels and 'y_pred' are your predicted labels
# If 'y_pred' is not yet defined, you would need to predict it using your model
y_pred = best_model.predict(X_test)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Assuming 'label_encoder' is the LabelEncoder instance used to encode your labels
# Use the inverse transform method to get the original class names
class_names = label_encoder.inverse_transform(np.arange(cm.shape[0]))

# Calculate error rates for each class
error_rate = (cm.sum(axis=1) - np.diag(cm)) / cm.sum(axis=1)

# Plot the error rate by class
plt.figure(figsize=(12, 8))
plt.bar(class_names, error_rate, color='salmon')
plt.xlabel('Class Names')
plt.ylabel('Error Rate')
plt.title('Error Rate by Class')
plt.xticks(rotation=90)  # Rotate the class names for better readability if needed
plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Generate the classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Convert the report to a DataFrame
report_df = pd.DataFrame(report).transpose()

# Replace index with class names for clarity, using label_encoder
class_names = label_encoder.inverse_transform(sorted(np.unique(y_test)))

# Exclude the 'accuracy' row and split the DataFrame into metrics and support
metrics_df = report_df.iloc[:-3, :-1]  # Exclude the 'support' column and the last 3 summary rows
support_df = report_df.iloc[:-3, -1:].astype(int)  # Keep only the 'support' column, convert to int
metrics_df.index = class_names
support_df.index = class_names

# Setup the matplotlib figure and axes for a side-by-side plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 10), gridspec_kw={'width_ratios': [3, 1]})

# Plot the heatmap for metrics
sns.heatmap(metrics_df, annot=True, fmt=".2f", cmap='Blues', ax=ax1, cbar=False, annot_kws={'size':18})
ax1.set_title('Precision, Recall, and F1-Score', fontsize=20)
ax1.set_ylabel('Classes', fontsize=18)
ax1.set_xlabel('Metrics', fontsize=18)
ax1.tick_params(axis='both', which='major', labelsize=16)

# Plot the heatmap (actually just annotated cells) for support
sns.heatmap(support_df, annot=True, fmt="d", cmap=None, ax=ax2, cbar=False, annot_kws={'size':18})
ax2.set_title('Support', fontsize=20)
ax2.set_xlabel('Metrics', fontsize=18)
ax2.tick_params(axis='y', which='major', labelsize=16)
ax2.set_yticks([]) 

plt.tight_layout()
plt.show()
plt.savefig('Precision_recall_F1_support_scores.jpg')

### Only for test purposes, do not run it

### Use this sample of code for future in Sprint 3

In [ ]:
# create dataset
X = df_2017[['concept:name - lag_2', 'concept:name - lag_1', 'concept:name']]
y = df_2017[['concept:name next']]

# Use LabelEncoder to convert string labels to numerical values
label_encoder = LabelEncoder()
X['concept:name - lag_1'] = label_encoder.fit_transform(X['concept:name - lag_1'])
X['concept:name - lag_2'] = label_encoder.fit_transform(X['concept:name - lag_2'])
X['concept:name'] = label_encoder.fit_transform(X['concept:name'])
y = label_encoder.fit_transform(y)

# configure the cross-validation procedure
cv_outer = KFold(n_splits=10, shuffle=False, random_state=None)
# enumerate splits
outer_results = list()
for train_ix, test_ix in cv_outer.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix, :], X.iloc[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]
    # configure the cross-validation procedure
    cv_inner = KFold(n_splits=3, shuffle=False, random_state=None)
    # define the model
    model = RandomForestClassifier(n_jobs=1)
    # define search
    search = GridSearchCV(model, param_grid=param_grid, scoring='accuracy', cv=cv_inner, refit=True)
    # execute search
    result = search.fit(X_train, y_train)
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_
    # evaluate model on the hold out dataset
    yhat = best_model.predict(X_test)
    # evaluate the model
    acc = accuracy_score(y_test, yhat)
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))

/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_9386/504684746.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['concept:name - lag_1'] = label_encoder.fit_transform(X['concept:name - lag_1'])
/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_9386/504684746.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['concept:name - lag_2'] = label_encoder.fit_transform(X['concept:name - lag_2'])
/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_9386/504684746.py:9: SettingWithCop

>acc=0.980, est=0.950, cfg={'max_depth': 19, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 21}
>acc=0.960, est=0.952, cfg={'max_depth': 19, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 21}
>acc=0.960, est=0.952, cfg={'max_depth': 19, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 19}
>acc=0.950, est=0.953, cfg={'max_depth': 18, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 21}
>acc=0.960, est=0.952, cfg={'max_depth': 18, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 19}
>acc=0.960, est=0.952, cfg={'max_depth': 19, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 20}
>acc=0.940, est=0.954, cfg={'max_depth': 18, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 21}
>acc=0.920, est=0.957, cfg={'max_depth': 19, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 20}
>acc=0.919, est=0.955, cfg={'max_depth': 18, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimat

In [ ]:
# Save the best model to a file
model_filename = 'random_forest_model.joblib'
joblib.dump(best_model, model_filename)

# Save the best model to a file using pickle
model_filename = 'random_forest_model.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(best_model, model_file)

In [ ]:
df_2017

,Unnamed: 0,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,...,predicted_time_to_next_event_seconds,elapsed_time_from_start,predicted_start_time,predicted_step,predicted_time_to_next_event_seconds_XG,concept:name - lag_1,concept:name - lag_2,concept:name - lag_3,concept:name next,predicted_next_event_rfc
3,3.0,1,0,22,2,701704,6,3,5,1,...,0 days 00:00:35.170000,33119,128,7,0 days 00:00:15,22.0,8.0,4.0,21.0,21
4,4.0,0,0,21,2,630431,3,4,5,1,...,0 days 00:00:00.010000,33126,127,6,0 days 00:00:05.500000,22.0,22.0,8.0,3.0,3
5,5.0,4,0,3,0,169679,1,5,5,1,...,2 days 01:50:47.740000,33133,126,0,4 days 22:54:39.812500,21.0,22.0,22.0,4.0,4
6,6.0,0,0,4,0,219156,1,6,7,1,...,0 days 00:00:00.040000,0,112,1,0 days 00:00:53.700001,3.0,21.0,22.0,8.0,8
7,7.0,4,0,8,0,130913,1,7,7,1,...,0 days 00:00:00.300000,49,113,3,0 days 00:00:11.200000,4.0,3.0,21.0,22.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201085,1201085.0,1,148,25,2,708416,0,1200970,11,1,...,0 days 00:00:00.020000,999899,44,5,0 days 00:01:00.799999,7.0,10.0,20.0,11.0,25
1201086,1201086.0,4,148,11,1,323681,1,1200971,11,1,...,0 days 00:00:00.010000,999900,45,5,0 days 00:01:17.500000,25.0,7.0,10.0,11.0,25
1201087,1201087.0,4,148,11,1,250417,1,1200972,11,1,...,0 days 00:00:00.010000,999901,43,5,0 days 00:01:17.500000,11.0,25.0,7.0,11.0,25
1201088,1201088.0,4,148,11,1,316489,1,1200973,11,1,...,0 days 00:00:00.010000,999902,47,5,0 days 00:01:17.500000,11.0,11.0,25.0,11.0,19
